In [1]:
!unzip '/content/drive/MyDrive/Gen AI Training/GenAI Case Study Data.zip'

Archive:  /content/drive/MyDrive/Gen AI Training/GenAI Case Study Data.zip
  inflating: Data/Queries/Queries.xlsx  
  inflating: Data/PDF Documents.zip  
  inflating: Data/Labelled Data/Labelled_Data_Set1.xlsx  
  inflating: Data/Labelled Data/Labelled_Data_Set2.xlsx  


In [2]:
!unzip '/content/Data/PDF Documents.zip'

Archive:  /content/Data/PDF Documents.zip
   creating: PDF Documents/Set1/
  inflating: PDF Documents/Set1/Black_Decker_AirCompresssor_ID_4700_Scanned.pdf  
  inflating: PDF Documents/Set1/Black_Decker_AirCompresssor_ID_WX-RP810_Scanned.pdf  
  inflating: PDF Documents/Set1/Black_Decker_AirCompresssor_Nil1_Editable.pdf  
  inflating: PDF Documents/Set1/Black_Decker_Blender_ID_5000_Editable.pdf  
  inflating: PDF Documents/Set1/Black_Decker_Blender_ID_bl2010bgc_Editable.pdf  
  inflating: PDF Documents/Set1/Black_Decker_Blender_ID_bl2010wg_Editable.pdf  
  inflating: PDF Documents/Set1/Black_Decker_Blender_Nil1_Editable.pdf  
  inflating: PDF Documents/Set1/Black_Decker_CoffeeGrinder_ID_cbm3_Editable.pdf  
  inflating: PDF Documents/Set1/Black_Decker_CoffeeGrinder_ID_series_cbg5_Edita.pdf  
  inflating: PDF Documents/Set1/Black_Decker_CoffeeGrinder_Nil1_Editable.pdf  
  inflating: PDF Documents/Set1/Black_Decker_CoffeeMaker_ID_am7_Editable.pdf  
  inflating: PDF Documents/Set1/Black_Dec

In [3]:
!pip install pymupdf PyPDF2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 11.2 MB/s eta 0:00:00


In [4]:
!pip install -U sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 259.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=51d016728ddcd0befe4b0b7f4f2d80ebcceb5fb6b5628baf777d83e55f565f11
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [5]:
!pip install faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 58.4 MB/s eta 0:00:00


In [6]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00


In [7]:
!pip install nltk rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=0b5cdd0af90f2c5c26be064f09e6b5d6325b804cf208371b762e741e4ec34650
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [8]:
# # !pip install llama-cpp-python
# !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
# !pip install llama-cpp-python==0.1.78
# !pip install numpy==1.23.4


In [9]:
# !pip install huggingface_hub

In [10]:
# from huggingface_hub import hf_hub_download
# model_path = hf_hub_download(repo_id='TheBloke/Llama-2-13B-chat-GGML', filename='llama-2-13b-chat.ggmlv3.q5_1.bin')



In [24]:
import os
from pathlib import Path
import fitz
import torch
import faiss
# from llama_cpp import Llama
from sentence_transformers import SentenceTransformer

from transformers import AutoModelForCausalLM, AutoTokenizer



def parse_pdf_files(folder_path, chunk_size, chunk_overlap):
  text_chunks = []
  for file_name in os.listdir(folder_path):
        if file_name.endswith('.pdf'):
          file_path = os.path.join(folder_path, file_name)
          pdf_document = fitz.open(file_path)
          num_pages = pdf_document.page_count
          for page_num in range(num_pages):
            page = pdf_document.load_page(page_num)
            text = page.get_text()

            # Split text into chunks with overlap
            for i in range(0, len(text), chunk_size - chunk_overlap):
                text_chunks.append(text[i:i + chunk_size])
          pdf_document.close()
  return text_chunks



def embedder(text_chunks):
    embed_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    embeddings = embed_model.encode(texts)
    return embeddings, embed_model


def save_embeddings(embeddings):
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    faiss.write_index(index, 'my_retriever.index')
    return index


def retrieve_context(query, embed_model, faiss_index, embeddings, texts):
    # llm_model_name = "databricks/dolly-v2-3b"
    llm_model_name ='bert-base-uncased'
    llm_tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
    llm_model = AutoModelForCausalLM.from_pretrained(llm_model_name)
    query_embedding = embed_model.encode([query])[0]
    D, I = faiss_index.search(query_embedding.reshape(1, -1), k=5)  # top 5 context similarity search
    similar_text = []
    for i in range(len(I[0])):
        similar_text_index = I[0][i]
        similar_text.append(texts[similar_text_index])

    # collect all similar texts for context
    context = " ".join(similar_text)

    # encode context using tokenizer
    inputs = llm_tokenizer.encode("Question: " + query + " Context: " + context, return_tensors="pt", max_length=512)

    # generate using LLM model
    with torch.no_grad():
      outputs = llm_model.generate(inputs, max_length=20)

    # deecode the output using tokenizer
    retrieved_answer = llm_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return retrieved_answer




folder_path = '/content/PDF Documents/Set1'
texts = parse_pdf_files(folder_path, chunk_size=500, chunk_overlap=50)





In [25]:
texts[-5:]

[' ground source see Figure "G".\nAttach the end of the positive wire to a dashboard illumination\nwire using a wire connector.  Slip the wire connector over the\nexisting dashboard illumination wire and insert the un-stripped\ngauge panel wire into the wire connector.  Close the wire connector\nover the wires with pliers see Figure "E".  Attach the black wire to\na ground source by crimping a ring connector on to the wire and\nsecuring it to a suitable ground source on the vehicle.  Note:  Should\nadditi',
 "round source on the vehicle.  Note:  Should\nadditional wire be necessary to reach the dashboard illumination\nwire and ground source, use 16 gage multi-strand wire.\nSTEP 9 - CHECK THE SYSTEM\nWith the Level Command kit and your air helper springs installed, you are ready to test the system.  Reattach the negative battery\ncable.  Turn on the vehicle's ignition.  Push the paddle switch up to inflate the air springs.  The gauge will display how much air\npressure is in the air spr

In [26]:
embeddings, embed_model = embedder(texts)
faiss_index = save_embeddings(embeddings)

In [27]:

query = "What to do after selecting a mounting location for the control panel?"
answers = retrieve_context(query, embed_model, faiss_index, embeddings, texts)


If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 512, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


In [28]:
answers

'question : what to do after selecting a mounting location for the control panel? context : ll mount assembly avoid accidentally switching the tool on during assembly. fitting the guard ( fig. b ) n remove the screw ( 15 ) from the guard ( 8 ). n place the guard onto the tool as shown. n firmly press the guard until it snaps into place. n secure the guard with the screw ( 15 ). never use the tool unless the guard is properly fitted. mounting the wall mount to the wall ( fig. c ) the wall mount ( 14 ) is used to provide a convenient storage and charging point for the tool. n find a suitable indoor ppliance. • allow at least 4 ” between the side opposite the control panel of your oven and adjacent wall or other appliance. you can mount the other appliances flush to one another. †. mountingparts ( part # sdc - mountingkit ) a. 6spacers ½ " ¼ " / 6 " / 6 " b. screws " ½ " ¼ " c. washers. mountingtemplate ( inbox ) youwillalsoneed : • tape • ruler ( oryoumaycutouttherul lock nuts washers ma

In [29]:
# import evaluate
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

import pandas as pd


def evaluate(query, answers):
    ground_truth = pd.read_excel("/content/Data/Labelled Data/Labelled_Data_Set1.xlsx")
    ground_truth = ground_truth[ground_truth['Queries']==query]['Answers'].iloc[0]

    predictions = answers.split('context :')[1]

    # bleu = evaluate.load("bleu")
    # rouge = evaluate.load('rouge')
    rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)


    # bleu_score = bleu.compute(predictions=predictions, references=ground_truth)
    bleu_score = sentence_bleu(ground_truth, predictions)
    rouge_score = rouge.score(ground_truth, predictions)['rougeL'].fmeasure

    return bleu_score, rouge_score


bleu_score, rouge_score = evaluate(query,answers)

print('BLEU Score :', bleu_score)
print('ROUGE Score :', rouge_score)

BLEU Score : 6.1044198792252615e-232
ROUGE Score : 0.16666666666666669


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_